Objective: Use the ABIDE dataset to explore a GAT for prediction of autism (imaging features only, no phenotypes). Leveraging CUDA on RTX 2050.

We're looking at a paper that proposed this architecture 

To fully utilize the phenotype information, this paper proposes a heterogeneous graph convolutional attention network (HCAN) model to classify ASD. By combining an attention mechanism and a heterogeneous graph convolutional network, important aggregated features can be extracted in the HCAN. The model consists of a multilayer HCAN feature extractor and a multilayer perceptron (MLP) classifier. First, a heterogeneous population graph was constructed based on the fMRI and phenotypic data. Then, a multilayer HCAN is used to mine graph-based features from the heterogeneous graph. Finally, the extracted features are fed into an MLP for the final classification.

But the thing is, we're looking at phenotypic data here


In [ ]:
# Graph Attention Network for ASD Classification (NO PHENOTYPIC DATA)
# Inspired by HCAN but using ONLY imaging-derived features

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool, global_max_pool
from torch_geometric.data import Data, DataLoader as PyGDataLoader
import numpy as np
import pandas as pd

# Check for CUDA/GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("   Note: Running on CPU - training will be slower")

## Graph Attention Network Architecture (Pure Imaging)

Instead of HCAN's heterogeneous graph with phenotypic nodes, we use:

### Our Approach: Homogeneous Graph with Attention

1. **Graph Construction**: Each subject = one graph
   - **Nodes**: ROIs (brain regions) 
   - **Edges**: Functional connectivity (correlation-based)
   - **Node Features**: Correlation vectors (purely fMRI-derived)

2. **Multi-Head Graph Attention Layers**:
   - Replace heterogeneous convolution with **GAT (Graph Attention)**
   - Learns which ROI connections are important
   - Multi-head attention for diverse feature extraction

3. **Hierarchical Feature Extraction**:
   - Layer 1: Local connectivity patterns
   - Layer 2: Higher-order brain network structure
   - Layer 3: Global integration features

4. **Readout + MLP Classifier**:
   - Aggregate graph-level features
   - MLP for final ASD vs Control classification

**Key Difference**: HCAN uses phenotype nodes in a population graph. We use **attention within each brain graph** instead!

In [ ]:
# Data Loading Functions (Fixed & Robust)
import glob
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch_geometric.data import Data

def load_timeseries_1d(path: str) -> np.ndarray:
    """Load .1D file as T×N array"""
    arr = np.loadtxt(path)
    return arr

def corr_matrix(timeseries: np.ndarray) -> np.ndarray:
    """Compute Pearson correlation matrix"""
    C = np.corrcoef(timeseries, rowvar=False)
    C = np.nan_to_num(C, nan=0.0)
    C = np.clip(C, -1.0, 1.0)
    return C

def build_sparse_edges_from_corr(C: np.ndarray, k: int = 20):
    """Create sparse edge_index via top-k correlations"""
    N = C.shape[0]
    np.fill_diagonal(C, 0.0)
    
    idx_src, idx_dst, weights = [], [], []
    absC = np.abs(C)
    k_eff = min(k, max(1, N - 1))
    
    for i in range(N):
        nbrs = np.argpartition(absC[i], -k_eff)[-k_eff:]
        for j in nbrs:
            if i != j:
                idx_src.append(i)
                idx_dst.append(j)
                weights.append(C[i, j])
    
    # Make undirected
    idx_all = np.concatenate([np.vstack([idx_src, idx_dst]), 
                              np.vstack([idx_dst, idx_src])], axis=1)
    w_all = np.array(weights + weights, dtype=np.float32)
    
    # Deduplicate
    pairs = set()
    uniq_src, uniq_dst, uniq_w = [], [], []
    for (s, d), w in zip(idx_all.T, w_all):
        key = (int(s), int(d))
        if key not in pairs and s != d:
            pairs.add(key)
            uniq_src.append(s)
            uniq_dst.append(d)
            uniq_w.append(w)
    
    edge_index = torch.tensor([uniq_src, uniq_dst], dtype=torch.long)
    edge_attr = torch.tensor(uniq_w, dtype=torch.float)
    return edge_index, edge_attr

def graph_from_timeseries(timeseries: np.ndarray, topk: int = 20):
    """Build PyG Data from fMRI timeseries"""
    C = corr_matrix(timeseries)
    N = C.shape[0]
    
    # Node features: correlation vectors (purely imaging-derived)
    x = torch.tensor(C, dtype=torch.float)
    
    # Sparse edges
    edge_index, edge_attr = build_sparse_edges_from_corr(C.copy(), k=topk)
    
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    return data

print("Data loading functions defined")

In [ ]:
# Multi-Head Graph Attention Network for ASD Classification
# NO PHENOTYPIC DATA - Pure imaging features only!

class MultiHeadGATLayer(nn.Module):
    """
    Multi-head Graph Attention Layer
    Replaces HCAN's heterogeneous conv with attention mechanism
    """
    def __init__(self, in_channels, out_channels, heads=8, dropout=0.3, concat=True):
        super().__init__()
        self.gat = GATConv(
            in_channels=in_channels,
            out_channels=out_channels,
            heads=heads,
            dropout=dropout,
            concat=concat,
            edge_dim=1  # Use edge weights (correlation values)
        )
        self.bn = nn.BatchNorm1d(out_channels * heads if concat else out_channels)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, edge_index, edge_attr=None):
        # Multi-head attention aggregation
        x = self.gat(x, edge_index, edge_attr=edge_attr)
        x = self.bn(x)
        x = F.elu(x)
        x = self.dropout(x)
        return x


class BrainGAT(nn.Module):
    """
    Graph Attention Network for Brain Connectivity Analysis
    
    Architecture (HCAN-inspired, NO phenotypic data):
    - Multi-layer GAT feature extractor (attention-based aggregation)
    - Readout: mean + max pooling
    - MLP classifier
    
    Key differences from HCAN:
    - Homogeneous graph (only ROI nodes, no phenotype nodes)
    - Pure imaging features (correlation-based)
    - Multi-head attention for learning important connections
    """
    def __init__(self, 
                 in_channels,
                 hidden_channels=256,
                 num_layers=3,
                 heads=8,
                 dropout=0.3,
                 num_classes=2):
        super().__init__()
        
        self.num_layers = num_layers
        
        # Multi-layer GAT feature extractor
        self.convs = nn.ModuleList()
        
        # First layer: in_channels -> hidden_channels
        self.convs.append(
            MultiHeadGATLayer(in_channels, hidden_channels, heads, dropout, concat=True)
        )
        
        # Middle layers: (hidden * heads) -> hidden_channels
        for _ in range(num_layers - 2):
            self.convs.append(
                MultiHeadGATLayer(hidden_channels * heads, hidden_channels, heads, dropout, concat=True)
            )
        
        # Last layer: average attention heads instead of concat
        self.convs.append(
            MultiHeadGATLayer(hidden_channels * heads, hidden_channels, heads, dropout, concat=False)
        )
        
        # Readout dimension (mean + max)
        self.readout_dim = hidden_channels * 2
        
        # MLP Classifier (like HCAN)
        self.classifier = nn.Sequential(
            nn.Linear(self.readout_dim, hidden_channels),
            nn.BatchNorm1d(hidden_channels),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.BatchNorm1d(hidden_channels // 2),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_channels // 2, num_classes)
        )
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        edge_attr = data.edge_attr if hasattr(data, 'edge_attr') else None
        batch = data.batch if hasattr(data, 'batch') else torch.zeros(x.size(0), dtype=torch.long, device=x.device)
        
        # Multi-layer GAT feature extraction
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index, edge_attr)
        
        # Graph-level readout: mean + max (captures global patterns)
        x_mean = global_mean_pool(x, batch)
        x_max = global_max_pool(x, batch)
        graph_features = torch.cat([x_mean, x_max], dim=-1)
        
        # MLP classification
        out = self.classifier(graph_features)
        
        return out
    
    def get_attention_weights(self, data):
        """
        Extract attention weights for interpretation
        Shows which ROI connections are important
        """
        x, edge_index = data.x, data.edge_index
        edge_attr = data.edge_attr if hasattr(data, 'edge_attr') else None
        
        attention_weights = []
        for conv in self.convs:
            # GATConv stores attention weights during forward pass
            x = conv.gat(x, edge_index, edge_attr, return_attention_weights=True)
            if isinstance(x, tuple):
                x, (edge_idx, attn) = x
                attention_weights.append((edge_idx, attn))
                x = conv.bn(x)
                x = F.elu(x)
        
        return attention_weights


print("BrainGAT architecture defined")
print("  - Multi-head attention: 8 heads per layer")
print("  - 3-layer hierarchical feature extraction")
print("  - NO phenotypic data - pure imaging features!")
print("  - Attention mechanism learns important brain connections")

In [ ]:
# Load ABIDE Dataset (Robust Loading)
from sklearn.model_selection import train_test_split

workspace_root = '/home/moew/Documents/ABIDE'
data_dir = os.path.join(workspace_root, 'abide_data/Outputs/cpac/nofilt_noglobal/rois_cc400/')
phenotype_file = os.path.join(workspace_root, 'Phenotypic_V1_0b_preprocessed1.csv')

def load_abide_graphs(data_dir, phenotype_file, topk=20):
    """Load ABIDE subjects as graphs (NO PHENOTYPIC FEATURES!)"""
    pheno_df = pd.read_csv(phenotype_file)
    roi_files = sorted(glob.glob(f'{data_dir}/*.1D'))
    
    print(f"Loading ABIDE data...")
    print(f"   Phenotype CSV: {len(pheno_df)} subjects")
    print(f"   .1D files found: {len(roi_files)}")
    print(f"   Using ONLY imaging features (no demographics!)")
    
    graphs, labels, subjects = [], [], []
    
    # Site mapping to handle naming discrepancies
    site_map = {
        'MaxMun': 'MAX_MUN',
        'Leuven_1': 'LEUVEN_1',
        'Leuven_2': 'LEUVEN_2',
        'UCLA_1': 'UCLA_1',
        'UCLA_2': 'UCLA_2',
        'UM_1': 'UM_1',
        'UM_2': 'UM_2',
        'Trinity': 'TRINITY',
        'Yale': 'YALE',
        'Olin': 'OLIN',
        'OHSU': 'OHSU',
        'SBL': 'SBL',
        'SDSU': 'SDSU',
        'Stanford': 'STANFORD',
        'Caltech': 'CALTECH',
        'CMU': 'CMU',
        'KKI': 'KKI',
        'NYU': 'NYU',
        'Pitt': 'PITT',
        'USM': 'USM'
    }
    
    for file_path in roi_files:
        try:
            filename = Path(file_path).stem
            parts = filename.replace('_rois_cc400', '').split('_')
            
            if len(parts) < 2:
                continue
            
            # Robust site parsing
            site = parts[0]
            subject_id_idx = 1
            
            # Check for multi-part site names (e.g., Leuven_1)
            if len(parts) > 2 and parts[1].isdigit() and len(parts[1]) == 1:
                 site = f"{parts[0]}_{parts[1]}"
                 subject_id_idx = 2
            
            # Map site name
            if site in site_map:
                site = site_map[site]
            elif site.upper() in site_map.values():
                site = site.upper()
                
            # Find subject ID (first numeric part after site)
            subject_id = None
            for part in parts[subject_id_idx:]:
                try:
                    subject_id = int(part)
                    break
                except ValueError:
                    continue
            
            if subject_id is None:
                continue
            
            # Match to phenotype for LABEL ONLY
            subject_row = pheno_df[
                (pheno_df['SITE_ID'] == site) & 
                (pheno_df['SUB_ID'] == subject_id)
            ]
            
            if not subject_row.empty:
                dx_group = subject_row['DX_GROUP'].values[0]
                
                if dx_group in [1, 2]:
                    ts = load_timeseries_1d(file_path)
                    graph = graph_from_timeseries(ts, topk=topk)
                    graph.y = torch.tensor([dx_group - 1], dtype=torch.long)
                    
                    graphs.append(graph)
                    labels.append(dx_group - 1)
                    subjects.append(f"{site}_{subject_id}")
        except Exception as e:
            continue
    
    print(f"\nLoaded {len(graphs)} subjects")
    print(f"  - ASD: {labels.count(1)} | Control: {labels.count(0)}")
    if graphs:
        print(f"  - Graph: {graphs[0].x.shape[0]} nodes, {graphs[0].edge_index.shape[1]} edges")
        print(f"  - Features: {graphs[0].x.shape[1]}-dim correlation vectors")
    
    return graphs, labels, subjects

# Load data with moderate connectivity (k=20)
graphs, labels, subjects = load_abide_graphs(data_dir, phenotype_file, topk=20)

# Stratified train/val/test split (70/15/15)
train_val_graphs, test_graphs, train_val_labels, test_labels = train_test_split(
    graphs, labels, test_size=0.15, random_state=42, stratify=labels
)

train_graphs, val_graphs, train_labels, val_labels = train_test_split(
    train_val_graphs, train_val_labels, test_size=0.176, random_state=42, stratify=train_val_labels
)

# Create DataLoaders
train_loader = PyGDataLoader(train_graphs, batch_size=16, shuffle=True)
val_loader = PyGDataLoader(val_graphs, batch_size=16, shuffle=False)
test_loader = PyGDataLoader(test_graphs, batch_size=16, shuffle=False)

print(f"\nData split: {len(train_graphs)}/{len(val_graphs)}/{len(test_graphs)} (train/val/test)")
print(f"Batch size: 16 (optimized for GPU)")
print(f"Ready for training!")

In [ ]:
# Debug Data Loading - Why are we losing subjects?

def debug_data_loading(data_dir, phenotype_file):
    print("🕵️‍♀️ Debugging Data Loading Process...")
    
    pheno_df = pd.read_csv(phenotype_file)
    roi_files = sorted(glob.glob(f'{data_dir}/*.1D'))
    
    print(f"   Total .1D files: {len(roi_files)}")
    print(f"   Total Phenotype records: {len(pheno_df)}")
    
    # Counters
    stats = {
        'loaded': 0,
        'filename_parse_error': 0,
        'no_subject_id': 0,
        'pheno_mismatch': 0,
        'invalid_dx': 0,
        'load_error': 0
    }
    
    # Check a few filenames to see format
    print(f"\n   Sample filenames:")
    for f in roi_files[:5]:
        print(f"   - {Path(f).name}")
        
    print(f"\n   Sample Phenotype Sites: {pheno_df['SITE_ID'].unique()[:5]}")
    
    for file_path in roi_files:
        try:
            filename = Path(file_path).stem
            # Attempt parsing
            parts = filename.replace('_rois_cc400', '').split('_')
            
            if len(parts) < 2:
                stats['filename_parse_error'] += 1
                continue
                
            site = parts[0]
            
            # Find subject ID
            subject_id = None
            for part in parts[1:]:
                try:
                    subject_id = int(part)
                    break
                except ValueError:
                    continue
            
            if subject_id is None:
                stats['no_subject_id'] += 1
                continue
            
            # Check matching
            # Try exact match first
            subject_row = pheno_df[
                (pheno_df['SITE_ID'] == site) & 
                (pheno_df['SUB_ID'] == subject_id)
            ]
            
            if subject_row.empty:
                # Try case insensitive site match
                subject_row = pheno_df[
                    (pheno_df['SITE_ID'].str.lower() == site.lower()) & 
                    (pheno_df['SUB_ID'] == subject_id)
                ]
            
            if subject_row.empty:
                stats['pheno_mismatch'] += 1
                # Print first few mismatches to debug
                if stats['pheno_mismatch'] <= 5:
                    print(f"   ❌ Mismatch: File={filename} -> Site={site}, ID={subject_id}")
                continue
                
            dx_group = subject_row['DX_GROUP'].values[0]
            if dx_group not in [1, 2]:
                stats['invalid_dx'] += 1
                continue
                
            stats['loaded'] += 1
            
        except Exception as e:
            stats['load_error'] += 1
            
    print(f"\n📊 Loading Statistics:")
    for k, v in stats.items():
        print(f"   {k:<20}: {v}")
        
    return stats

# Run debug
debug_stats = debug_data_loading(data_dir, phenotype_file)

## Ultra Low Memory Mode (RTX 2050 - 4GB VRAM)

For RTX 2050 with 4GB VRAM, use this ultra-optimized configuration:

### Memory Optimizations:
- **Batch size 4**: Minimal memory footprint
- **Smaller hidden**: 64 channels
- **Fewer attention heads**: 4 heads
- **Empty cache**: Clear GPU memory between batches

### Memory Usage:
- This config (batch=4, hidden=64): ~1.5-2GB VRAM (safe)

### Performance Impact:
- Training time: ~4-5 min/epoch
- Accuracy: Should still reach ~60-70% (data-limited problem)

In [ ]:
# Ultra Low Memory Configuration for RTX 2050
# Includes Early Stopping & Checkpointing

# Get input dimensions from data
sample_graph = train_graphs[0]
in_features = sample_graph.x.size(1)  # 392 for CC400

# Clear GPU memory first
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Cleared GPU cache")

print("\nUltra Low Memory Mode")
print("   This configuration uses minimal VRAM (~1.5-2GB)")

# Tiny model: 64 hidden channels, 4 attention heads
model_ultra = BrainGAT(
    in_channels=in_features,
    hidden_channels=64,   # Reduced from 128 → 64
    num_layers=3,
    heads=4,              # Reduced from 8 → 4
    dropout=0.3,
    num_classes=2
).to(device)

ultra_params = sum(p.numel() for p in model_ultra.parameters())
print(f"\nUltra Low Memory Model:")
print(f"   Parameters: {ultra_params:,}")
print(f"   Memory: ~{ultra_params * 2 / 1e6:.0f} MB (FP16)")

# Smallest possible batch size
print(f"\nUltra small batches:")
train_loader_ultra = PyGDataLoader(train_graphs, batch_size=4, shuffle=True)
val_loader_ultra = PyGDataLoader(val_graphs, batch_size=4, shuffle=False)
test_loader_ultra = PyGDataLoader(test_graphs, batch_size=4, shuffle=False)

print(f"   Batch size: 4 (absolute minimum)")
print(f"   Train batches: {len(train_loader_ultra)}")

# --- Checkpointing & Early Stopping Classes ---
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

class ModelCheckpoint:
    def __init__(self, filepath='models/best_model.pth'):
        self.filepath = filepath
        self.best_acc = 0.0
        os.makedirs(os.path.dirname(filepath), exist_ok=True)

    def __call__(self, model, val_acc, epoch):
        if val_acc > self.best_acc:
            self.best_acc = val_acc
            torch.save(model.state_dict(), self.filepath)
            print(f"   ✓ New best model saved! (Acc: {val_acc:.2f}%)")
        
        # Also save latest checkpoint for crash recovery
        latest_path = self.filepath.replace('best_model', 'latest_checkpoint')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_acc': self.best_acc
        }, latest_path)

# Setup Training
optimizer = torch.optim.Adam(model_ultra.parameters(), lr=0.001, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()
early_stopping = EarlyStopping(patience=15, min_delta=0.001)
checkpoint = ModelCheckpoint(filepath='models/braingat_ultra_best.pth')

print(f"\n🚀 Training with ultra low memory settings...")
print(f"   Batch size: 4 (smallest possible)")
print(f"   Expected: ~4-5 min/epoch")
print(f"   This WILL work on 4GB VRAM!")

# Training Loop
epochs = 100
for epoch in range(1, epochs + 1):
    model_ultra.train()
    total_loss = 0
    correct = 0
    total = 0
    
    for data in train_loader_ultra:
        data = data.to(device)
        optimizer.zero_grad()
        out = model_ultra(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * data.num_graphs
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())
        total += data.num_graphs
        
    train_loss = total_loss / total
    train_acc = 100. * correct / total
    
    # Validation
    model_ultra.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for data in val_loader_ultra:
            data = data.to(device)
            out = model_ultra(data)
            loss = criterion(out, data.y)
            val_loss += loss.item() * data.num_graphs
            pred = out.argmax(dim=1)
            val_correct += int((pred == data.y).sum())
            val_total += data.num_graphs
    
    val_loss /= val_total
    val_acc = 100. * val_correct / val_total
    
    print(f"Epoch {epoch:03d}: Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%, "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
    
    # Checkpointing
    checkpoint(model_ultra, val_acc, epoch)
    
    # Early Stopping
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("⏹️ Early stopping triggered!")
        break
        
print("✅ Ultra low memory training complete!")

In [ ]:
# Train Ultra Low Memory Model

print("Training with ultra low memory settings...")
print("   Batch size: 4 (smallest possible)")
print("   Expected: ~4-5 min/epoch")
print()

# Custom training function with aggressive memory management
def train_ultra_low_memory(model, train_loader, val_loader, num_epochs=100, lr=0.0005, patience=20):
    """Training with aggressive memory management for low VRAM"""
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=7, verbose=True
    )
    criterion = nn.CrossEntropyLoss()
    
    use_amp = device.type == 'cuda'
    scaler = torch.cuda.amp.GradScaler() if use_amp else None
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': [],
        'best_val_acc': 0.0, 'best_epoch': 0
    }
    
    patience_counter = 0
    
    print(f"\n{'='*70}")
    print(f"Ultra Low Memory Training")
    print(f"{'='*70}")
    print(f"Device: {device}")
    print(f"Memory optimization: MAXIMUM")
    print(f"{'='*70}")
    print(f"{'Epoch':<8}{'Train Loss':<12}{'Train Acc':<12}{'Val Loss':<12}{'Val Acc':<12}{'Status'}")
    print(f"{'='*70}")
    
    for epoch in range(num_epochs):
        # === TRAINING ===
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_idx, batch in enumerate(train_loader):
            batch = batch.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with torch.cuda.amp.autocast():
                    out = model(batch)
                    loss = criterion(out, batch.y.squeeze())
                
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                out = model(batch)
                loss = criterion(out, batch.y.squeeze())
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            train_loss += loss.item() * batch.num_graphs
            pred = out.argmax(dim=1)
            train_correct += (pred == batch.y.squeeze()).sum().item()
            train_total += batch.num_graphs
            
            # Clear cache every 10 batches to prevent fragmentation
            if use_amp and batch_idx % 10 == 0:
                torch.cuda.empty_cache()
        
        train_loss /= train_total
        train_acc = train_correct / train_total * 100
        
        # === VALIDATION ===
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                
                if use_amp:
                    with torch.cuda.amp.autocast():
                        out = model(batch)
                        loss = criterion(out, batch.y.squeeze())
                else:
                    out = model(batch)
                    loss = criterion(out, batch.y.squeeze())
                
                val_loss += loss.item() * batch.num_graphs
                pred = out.argmax(dim=1)
                val_correct += (pred == batch.y.squeeze()).sum().item()
                val_total += batch.num_graphs
        
        # Clear cache after validation
        if use_amp:
            torch.cuda.empty_cache()
        
        val_loss /= val_total
        val_acc = val_correct / val_total * 100
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        scheduler.step(val_acc)
        
        status = ""
        if val_acc > history['best_val_acc']:
            history['best_val_acc'] = val_acc
            history['best_epoch'] = epoch
            torch.save(model.state_dict(), 'braingat_ultra.pth')
            patience_counter = 0
            status = "Best"
        else:
            patience_counter += 1
            status = f"({patience_counter}/{patience})"
        
        if (epoch + 1) % 5 == 0 or epoch == 0 or patience_counter == 0 or patience_counter >= patience:
            print(f"{epoch+1:<8}{train_loss:<12.4f}{train_acc:<12.2f}{val_loss:<12.4f}{val_acc:<12.2f}{status}")
        
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            break
    
    model.load_state_dict(torch.load('braingat_ultra.pth'))
    
    print(f"{'='*70}")
    print(f"Training complete!")
    print(f"  Best val accuracy: {history['best_val_acc']:.2f}% (epoch {history['best_epoch']+1})")
    print(f"{'='*70}")
    
    return model, history

# Train the ultra low memory model
trained_model_ultra, train_history_ultra = train_ultra_low_memory(
    model_ultra,
    train_loader_ultra,
    val_loader_ultra,
    num_epochs=100,
    lr=0.0005,
    patience=20
)

print("\nUltra low memory training complete!")
print(f"Model saved to: braingat_ultra.pth")

In [ ]:
# Evaluate Ultra Low Memory Model

# Define evaluation function
def evaluate_gat(model, test_loader, device):
    """Comprehensive test set evaluation"""
    from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    model.eval()
    model = model.to(device)
    
    all_preds, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            out = model(batch)
            
            probs = F.softmax(out, dim=1)
            preds = out.argmax(dim=1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch.y.squeeze().cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
    
    accuracy = (np.array(all_preds) == np.array(all_labels)).mean() * 100
    auc = roc_auc_score(all_labels, all_probs)
    
    print("\n" + "="*70)
    print("TEST SET EVALUATION (Held-Out Data)")
    print("="*70)
    print(f"\nTest Accuracy: {accuracy:.2f}%")
    print(f"ROC AUC Score: {auc:.4f}")
    print(f"\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['Control', 'ASD'], digits=4))
    
    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Control', 'ASD'],
                yticklabels=['Control', 'ASD'])
    plt.title(f'BrainGAT Test Confusion Matrix\nAccuracy: {accuracy:.2f}%, AUC: {auc:.4f}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()
    
    return {
        'accuracy': accuracy,
        'auc': auc,
        'predictions': all_preds,
        'labels': all_labels,
        'probabilities': all_probs
    }

print("Evaluating ultra low memory model...")
test_results_ultra = evaluate_gat(trained_model_ultra, test_loader_ultra, device)

# Final summary
print("\n" + "="*70)
print("ULTRA LOW MEMORY PERFORMANCE")
print("="*70)
print(f"\nConfiguration:")
print(f"   Hidden channels: 64 (smallest)")
print(f"   Attention heads: 4 (reduced)")
print(f"   Parameters: {ultra_params:,}")
print(f"   Batch size: 4 (minimum)")
print(f"\nPerformance:")
print(f"   Test Accuracy: {test_results_ultra['accuracy']:.2f}%")
print(f"   ROC-AUC: {test_results_ultra['auc']:.4f}")
print("="*70)

## Improved Training: More Epochs + Elastic Net Regularization

Let's train properly with:

### Key Improvements:
1. **More Epochs**: 200 epochs (vs 100) - model needs more time
2. **Elastic Net**: L1 + L2 regularization (prevents overfitting)
3. **Lower Learning Rate**: 0.0001 (vs 0.0005) - more stable convergence
4. **Longer Patience**: 30 epochs (vs 20) - don't give up too early
5. **Stronger Dropout**: 0.4 (vs 0.3) - better generalization

### Why This Helps:
- Previous run: 85% train, 59% val → massive overfitting
- Elastic net (L1+L2) penalizes large weights and promotes sparsity
- Lower LR prevents getting stuck in bad local minima
- More epochs gives model time to find better solutions

### Expected Time:
- ~5 min/epoch × 60-80 epochs = 5-7 hours
- Will early stop when converged (won't run all 200 epochs)

In [ ]:
# Create Improved Model with Stronger Regularization

print("Creating improved model with elastic net regularization...")

# Clear memory first
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("Cleared GPU cache")

# Model with STRONGER dropout for better regularization
model_improved = BrainGAT(
    in_channels=in_features,
    hidden_channels=64,   # Same as ultra
    num_layers=3,
    heads=4,              # Same as ultra
    dropout=0.4,          # INCREASED from 0.3 → 0.4
    num_classes=2
).to(device)

improved_params = sum(p.numel() for p in model_improved.parameters())
print(f"\nImproved Model Configuration:")
print(f"   Parameters: {improved_params:,}")
print(f"   Dropout: 0.4 (stronger regularization)")
print(f"   Ready for elastic net training!")
print(f"\nModel initialized successfully")

In [ ]:
# Training with Elastic Net (L1 + L2) Regularization

def train_with_elastic_net(model, train_loader, val_loader, 
                           num_epochs=200, lr=0.0001, 
                           l1_lambda=1e-5, l2_lambda=5e-4,
                           patience=30):
    """
    Training with Elastic Net regularization (L1 + L2)
    
    Elastic Net = L1 (Lasso) + L2 (Ridge)
    - L1: Promotes sparsity (sets some weights to zero)
    - L2: Prevents large weights (smoothness)
    - Together: Better generalization than either alone!
    """
    model = model.to(device)
    
    # AdamW for L2, we'll add L1 manually
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=l2_lambda)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=10, verbose=True, min_lr=1e-6
    )
    criterion = nn.CrossEntropyLoss()
    
    use_amp = device.type == 'cuda'
    scaler = torch.cuda.amp.GradScaler() if use_amp else None
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_acc': [], 'val_acc': [],
        'learning_rates': [],
        'best_val_acc': 0.0, 'best_epoch': 0
    }
    
    patience_counter = 0
    
    print(f"\n{'='*80}")
    print(f"Training with Elastic Net Regularization")
    print(f"{'='*80}")
    print(f"Device: {device}")
    print(f"Learning rate: {lr}")
    print(f"L1 lambda: {l1_lambda} (sparsity)")
    print(f"L2 lambda: {l2_lambda} (smoothness)")
    print(f"Dropout: 0.4 (strong regularization)")
    print(f"Patience: {patience} epochs")
    print(f"Max epochs: {num_epochs}")
    print(f"{'='*80}")
    print(f"{'Epoch':<8}{'Train Loss':<12}{'Train Acc':<12}{'Val Loss':<12}{'Val Acc':<12}{'LR':<12}{'Status'}")
    print(f"{'='*80}")
    
    for epoch in range(num_epochs):
        # === TRAINING ===
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_idx, batch in enumerate(train_loader):
            batch = batch.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with torch.cuda.amp.autocast():
                    out = model(batch)
                    loss = criterion(out, batch.y.squeeze())
                    
                    # Add L1 regularization (manually)
                    l1_norm = sum(p.abs().sum() for p in model.parameters())
                    loss = loss + l1_lambda * l1_norm
                
                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
            else:
                out = model(batch)
                loss = criterion(out, batch.y.squeeze())
                
                # Add L1 regularization
                l1_norm = sum(p.abs().sum() for p in model.parameters())
                loss = loss + l1_lambda * l1_norm
                
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
            
            train_loss += loss.item() * batch.num_graphs
            pred = out.argmax(dim=1)
            train_correct += (pred == batch.y.squeeze()).sum().item()
            train_total += batch.num_graphs
            
            # Clear cache periodically
            if use_amp and batch_idx % 15 == 0:
                torch.cuda.empty_cache()
        
        train_loss /= train_total
        train_acc = train_correct / train_total * 100
        
        # === VALIDATION ===
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                
                if use_amp:
                    with torch.cuda.amp.autocast():
                        out = model(batch)
                        loss = criterion(out, batch.y.squeeze())
                else:
                    out = model(batch)
                    loss = criterion(out, batch.y.squeeze())
                
                val_loss += loss.item() * batch.num_graphs
                pred = out.argmax(dim=1)
                val_correct += (pred == batch.y.squeeze()).sum().item()
                val_total += batch.num_graphs
        
        if use_amp:
            torch.cuda.empty_cache()
        
        val_loss /= val_total
        val_acc = val_correct / val_total * 100
        
        # Get current learning rate
        current_lr = optimizer.param_groups[0]['lr']
        
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['learning_rates'].append(current_lr)
        
        # Learning rate scheduling
        scheduler.step(val_acc)
        
        status = ""
        if val_acc > history['best_val_acc']:
            history['best_val_acc'] = val_acc
            history['best_epoch'] = epoch
            torch.save(model.state_dict(), 'braingat_elastic_net.pth')
            patience_counter = 0
            status = "Best"
        else:
            patience_counter += 1
            status = f"({patience_counter}/{patience})"
        
        # Print every 5 epochs or important events
        if (epoch + 1) % 5 == 0 or epoch == 0 or patience_counter == 0 or patience_counter >= patience:
            print(f"{epoch+1:<8}{train_loss:<12.4f}{train_acc:<12.2f}{val_loss:<12.4f}{val_acc:<12.2f}{current_lr:<12.2e}{status}")
        
        if patience_counter >= patience:
            print(f"\nEarly stopping at epoch {epoch+1}")
            print(f"   No improvement for {patience} epochs")
            break
    
    # Load best model
    model.load_state_dict(torch.load('braingat_elastic_net.pth'))
    
    print(f"{'='*80}")
    print(f"Training complete!")
    print(f"  Best validation accuracy: {history['best_val_acc']:.2f}% (epoch {history['best_epoch']+1})")
    print(f"  Total epochs trained: {len(history['train_loss'])}")
    print(f"{'='*80}")
    
    return model, history

print("Elastic net training function defined")

In [ ]:
# Train Improved Model with Elastic Net

print("Starting long training run with elastic net regularization...")
print("   This will take 5-7 hours but should achieve better results!")
print()
print("What's happening:")
print("   - L1 regularization: Promoting sparse weights (feature selection)")
print("   - L2 regularization: Preventing large weights (smoothness)")
print("   - Dropout 0.4: Strong generalization")
print("   - Lower LR (0.0001): More stable convergence")
print("   - Patience 30: Won't give up too easily")
print()

# Train with elastic net
trained_model_improved, history_improved = train_with_elastic_net(
    model_improved,
    train_loader_ultra,
    val_loader_ultra,
    num_epochs=200,       # More epochs
    lr=0.0001,            # Lower learning rate
    l1_lambda=1e-5,       # L1 regularization strength
    l2_lambda=5e-4,       # L2 regularization strength
    patience=30           # Longer patience
)

print("\nLong training complete!")
print(f"Best model saved to: braingat_elastic_net.pth")
print(f"Best validation accuracy: {history_improved['best_val_acc']:.2f}%")
print(f"Total epochs: {len(history_improved['train_loss'])}")
print(f"\nReady for test evaluation!")

In [ ]:
# Evaluate Improved Model with Elastic Net

print("Evaluating improved model with elastic net...")
test_results_improved = evaluate_gat(trained_model_improved, test_loader_ultra, device)

# Comprehensive comparison
print("\n" + "="*80)
print("FINAL PERFORMANCE COMPARISON")
print("="*80)

print(f"\n{'Model':<30}{'Val Acc':<15}{'Test Acc':<15}{'Test AUC':<15}{'Epochs'}")
print("-"*80)
print(f"{'Ultra (quick, 22 epochs)':<30}{train_history_ultra['best_val_acc']:<15.2f}{test_results_ultra['accuracy']:<15.2f}{test_results_ultra['auc']:<15.4f}{len(train_history_ultra['train_loss'])}")
print(f"{'Improved (elastic net)':<30}{history_improved['best_val_acc']:<15.2f}{test_results_improved['accuracy']:<15.2f}{test_results_improved['auc']:<15.4f}{len(history_improved['train_loss'])}")

improvement = test_results_improved['accuracy'] - test_results_ultra['accuracy']
print(f"\n{'Improvement:':<30}{'':<15}{improvement:+.2f}%")

print("\n" + "="*80)
print("Analysis:")
print("="*80)

if test_results_improved['accuracy'] >= 65:
    print("EXCELLENT! Achieved 65%+ accuracy (expected range for pure imaging)")
    print("   This is competitive with published ASD classification papers!")
elif test_results_improved['accuracy'] >= 60:
    print("GOOD! Achieved 60-65% accuracy (solid performance)")
    print("   Better than random chance, meaningful signal detected!")
elif test_results_improved['accuracy'] >= 55:
    print("FAIR performance (55-60%). Model learned something but not optimal.")
else:
    print("Below expectations. Model struggled to learn patterns.")

print(f"\nKey Insights:")
print(f"   - Train-val gap at best epoch: {history_improved['train_acc'][history_improved['best_epoch']] - history_improved['best_val_acc']:.2f}%")
if history_improved['train_acc'][history_improved['best_epoch']] - history_improved['best_val_acc'] < 10:
    print(f"     Good generalization (gap < 10%)")
else:
    print(f"     Still some overfitting (gap > 10%)")

print(f"   - Learning rate at end: {history_improved['learning_rates'][-1]:.2e}")
print(f"   - Patience used: {len(history_improved['train_loss']) - history_improved['best_epoch'] - 1}/30")

print("\n" + "="*80)

## Troubleshooting OOM Errors

If you still get Out of Memory errors:

### 1. Restart Kernel First
Restart the kernel to clear all GPU memory, then run ONLY the ultra low memory cells.

### 2. Close Other Applications
- Close Chrome/Firefox (huge VRAM usage)
- Close other Python notebooks
- Stop any GPU-using processes: `nvidia-smi` to check

### 3. Set Environment Variable
```python
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
# Run this BEFORE creating any models
```

### 4. Even Smaller Batch Size (batch=2)
```python
# Absolute minimum - only if batch=4 still fails
train_loader_minimal = PyGDataLoader(train_graphs, batch_size=2, shuffle=True)
val_loader_minimal = PyGDataLoader(val_graphs, batch_size=2, shuffle=False)
```

### 5. CPU Fallback (Last Resort)
```python
# If GPU absolutely won't work, train on CPU
device = torch.device('cpu')
# Expect: ~10-15 min/epoch instead of 4-5 min
# But it WILL work and achieve same accuracy!
```

### 6. Monitor Memory Usage
```python
# Check GPU memory during training
import torch
if torch.cuda.is_available():
    print(f"Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
    print(f"Cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")
    print(f"Max allocated: {torch.cuda.max_memory_allocated()/1e9:.2f} GB")
```

## Summary: BrainGAT vs HCAN Comparison

### HCAN (Original Paper)
- Heterogeneous graph: ROI nodes + phenotype nodes
- Uses demographic features (age, sex, site, IQ)
- Population-level graph across subjects
- Risk of data leakage from phenotypic features

### Our BrainGAT (Pure Imaging)
- Homogeneous graph: ROI nodes only
- NO phenotypic data - only fMRI connectivity
- Subject-level graphs (proper data splitting)
- Multi-head attention for learning important connections
- GPU-optimized with mixed precision training

### Key Advantages
1. **No Data Leakage**: Pure imaging features only
2. **Interpretable**: Attention weights show important connections
3. **GPU-Optimized**: Faster training on RTX 2050/CUDA
4. **Scientifically Rigorous**: Proper train/val/test splits

### Expected Performance
- **HCAN (with phenotype)**: ~75-85% accuracy
- **BrainGAT (imaging only)**: ~65-75% accuracy (realistic!)
- Lower accuracy is EXPECTED without phenotypic shortcuts
- Still competitive with proper methodology!